In [12]:
!pip install tensorflow
!pip install torch torchvision

For this activity, you will use the CIFAR-10 dataset, which consists of 60,000 32 × 32 color images across 10 different classes. You will implement the same neural network architecture using both TensorFlow and PyTorch.

In [13]:

#load the dataset with tensorflow(keras)
import tensorflow as tf

# Load CIFAR-10 dataset
(train_images, train_activityels), (test_images, test_activityels) = tf.keras.datasets.cifar10.load_data()

# Normalize the images to a range of 0 to 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [14]:
#loading with pytorch
import torch
import torchvision
import torchvision.transforms as transforms

# Define a transformation to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Explanation
In both TensorFlow and PyTorch, the images are normalized so that pixel values range from 0 to 1 in TensorFlow and from –1 to 1 in PyTorch (using transforms.Normalize).

In PyTorch, we define a DataLoader to handle the batching and shuffling of the dataset.

Step 3: Define the neural network architecture
For both frameworks, we will define the same neural network architecture: a simple convolutional neural network (CNN) consisting of two convolutional layers followed by two fully connected layers.

In [15]:
#Define the CNN in TensorFlow (Keras)
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
#Define the CNN in PyTorch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Both architectures consist of two convolutional layers followed by pooling, flattening, and fully connected layers.

In TensorFlow, the input shape is explicitly defined, whereas PyTorch calculates the shape dynamically during the forward pass.

In [17]:
# Compile the model in tensorflow
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
#define the optimizer in pytorch
import torch.optim as optim

# Create an instance of the SimpleCNN model
pytorch_model = SimpleCNN()

# Define the optimizer and loss function using the PyTorch model instance
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Step 5: Training the neural network

In [19]:
# Train the model for 10 epochs in tensorflow
model.fit(train_images, train_activityels, epochs=10, batch_size=32, validation_data=(test_images, test_activityels))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 76s 46ms/step - accuracy: 0.3695 - loss: 1.7333 - val_accuracy: 0.5510 - val_loss: 1.2789
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.5757 - loss: 1.1999 - val_accuracy: 0.6120 - val_loss: 1.1157
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 45ms/step - accuracy: 0.6387 - loss: 1.0320 - val_accuracy: 0.6576 - val_loss: 0.9912
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 45ms/step - accuracy: 0.6694 - loss: 0.9412 - val_accuracy: 0.6494 - val_loss: 1.0003
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 46ms/step - accuracy: 0.6989 - loss: 0.8724 - val_accuracy: 0.6836 - val_loss: 0.9350
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 45ms/step - accuracy: 0.7144 - loss: 0.8273 - val_accuracy: 0.6789 - val_loss: 0.9370
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.7303 - loss: 0.7751 - val_accuracy: 0.6879 - val_loss: 0.9117
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.7421 -

In [21]:
# Training loop for PyTorch
# Training loop for PyTorch
for epoch in range(10):
    running_loss = 0.0
    for inputs, activityels in trainloader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = pytorch_model(inputs)  # Forward pass using the PyTorch model
        loss = criterion(outputs, activityels)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimize
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

Epoch 1, Loss: 1.4010555775823001
Epoch 1, Loss: 1.4010555775823001
Epoch 2, Loss: 1.033403793436857
Epoch 2, Loss: 1.033403793436857
Epoch 3, Loss: 0.8939307762580427
Epoch 3, Loss: 0.8939307762580427
Epoch 4, Loss: 0.8112368592259522
Epoch 4, Loss: 0.8112368592259522
Epoch 5, Loss: 0.7418212437011916
Epoch 5, Loss: 0.7418212437011916
Epoch 6, Loss: 0.6829046588903501
Epoch 6, Loss: 0.6829046588903501
Epoch 7, Loss: 0.637353381312435
Epoch 7, Loss: 0.637353381312435
Epoch 8, Loss: 0.5932897175265975
Epoch 8, Loss: 0.5932897175265975
Epoch 9, Loss: 0.5581374208342167
Epoch 9, Loss: 0.5581374208342167
Epoch 10, Loss: 0.520843729901146
Epoch 10, Loss: 0.520843729901146


Explanation
TensorFlow simplifies the training process using the .fit() method, while PyTorch requires manual loops to handle the forward pass, backpropagation, and optimization steps.

Step 6: Evaluate the model
Evaluate the model in TensorFlow

In [22]:
# Evaluate the model in tensorflow
test_loss, test_acc = model.evaluate(test_images, test_activityels)
print(f'Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.6890 - loss: 0.9405
Test accuracy: 0.6883999705314636


In [25]:
#evaluate in pytorch
correct = 0
total = 0
with torch.no_grad():
    for inputs, activityels in testloader:
        # Use the PyTorch model for prediction
        outputs = pytorch_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += activityels.size(0)
        correct += (predicted == activityels).sum().item()

print(f'Test accuracy: {100 * correct / total}%')

Test accuracy: 71.16%


Step 7: Compare and analyze
After training both models, compare their performance:

Accuracy: Compare the test accuracy of both models.

Ease of use: Which framework was easier to implement? Did TensorFlow’s higher-level API make implementation smoother, or did PyTorch's flexibility offer better control?

Debugging: Reflect on the debugging process. Did PyTorch’s dynamic graph provide better insight, or did TensorFlow’s simplified process make it easier?

Deliverables
At the end of this activity, you should have:

Your complete code for implementing and training the neural networks in both TensorFlow and PyTorch.

A comparison report discussing:

The test accuracy of both models.

The ease of implementation in both frameworks.

Key differences, including which framework you preferred and why.

Conclusion
By completing this activity, you’ve gained practical experience with two of the most popular deep learning frameworks. You’ve also compared how they differ in terms of implementation, ease of use, and performance. This will help you make informed decisions about which framework to use for your projects in the future.

Completed